In [1]:
%load_ext dotenv
%dotenv

In [2]:
from VLA_LLM import prompts
from VLA_LLM.agents import ChatConversationalVLAAgent
from VLA_LLM.tools import AppointmentAvailabilityTool
from VLA_LLM.tools import AppointmentSchedulerTool
from VLA_LLM.tools import AppointmentSchedulerAndAvailabilityTool
from VLA_LLM.tools import CurrentTimeTool
from VLA_LLM.community_info import EXAMPLE_COMMUNITY_DATA
from VLA_LLM.community_info import community_dict_to_prompt

In [3]:
# most deterministic results
temperature = 0

In [4]:
# tools = [CurrentTimeTool(), AppointmentSchedulerAndAvailabilityTool()]
tools = [CurrentTimeTool(), AppointmentSchedulerTool(), AppointmentAvailabilityTool()]

In [5]:
community_info = community_dict_to_prompt(EXAMPLE_COMMUNITY_DATA)

In [6]:
print(community_info)

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available

Pet policy: Pets Allowed. Extra fee for pets: 50.0


In [7]:
# from langchain.prompts import PromptTemplate
# prompt = PromptTemplate.from_template(f"{prompts.prompt_2}\n\n{{prospect_message}}")
# prompt.format(community_info=community_info)

In [8]:
prompt_template = (
    f"{prompts.prompt_three_tool_explicit.format(community_info=community_info)}\n\n"
    "Here is the prospect message:\n\n{prospect_message}"
)

In [9]:
print(prompt_template)

You are a leasing agent at a large multifamily apartment building talking with a prospect interested in renting one of your units. Your job is to answer questions about the community based on the information below. You should also try to gather their preferences like budget, move-in date, and desired layout, while nudging them to book a tour.

Here is information about the property that you can answer questions about:

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available

Pet policy: Pets Allowed. Extra fee for pets: 50.0

If the prospect provides an exact appointment time, first call the current time tool to get the current day of week and time. Then convert the appointment time into YYYY-MM-DD HH:MM:SS format with respect to the current day and time. After converting the appointment time into the correct format, pass it into the appointment scheduler tool and pa

In [10]:
agent = ChatConversationalVLAAgent(temperature, tools)

In [11]:
def run_agent(message):
    return agent.agent_chain.run(input=prompt_template.format(prospect_message=message))

### Test Conversation 1

Basic questions about the community.

In [12]:
run_agent("What is the parking situation?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The parking options at our apartment building include parking available for a fee of $100 per month. Would you like to know more about our parking options?"
}

> Finished chain.


'The parking options at our apartment building include parking available for a fee of $100 per month. Would you like to know more about our parking options?'

In [13]:
run_agent("i have a dog - can i bring him?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: Our pet policy allows for pets, including dogs. There is an extra fee of $50 for pets. Is there anything else I can assist you with?

In [14]:
run_agent("What is the parking situation? can i bring my dog as well?")



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "The parking situation at our apartment building is that we have parking available for a fee of $100 per month. As for pets, we do allow them, but there is an extra fee of $50 for pets. Let me know if you have any other questions or if you would like to schedule a tour!"
}

> Finished chain.


'The parking situation at our apartment building is that we have parking available for a fee of $100 per month. As for pets, we do allow them, but there is an extra fee of $50 for pets. Let me know if you have any other questions or if you would like to schedule a tour!'

### Test Conversation 2

Asking about an available appointment date.

In [15]:
run_agent("Can I schedule a tour for 7/27?")



> Entering new AgentExecutor chain...
{
    "action": "appointment_availability",
    "action_input": "2022-07-27"
}
Observation: There are no available times on the requested date.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize, but there are no available times for a tour on July 27th. Is there another date that works for you?"
}

> Finished chain.


'I apologize, but there are no available times for a tour on July 27th. Is there another date that works for you?'

In [17]:
run_agent("Can I book one for 11am?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Wednesday 2023-07-26 11:12:30
Thought:{
    "action": "appointment_scheduler_availability",
    "action_input": "2023-07-27 11:00:00"
}
Observation: Scheduled successfully for 11:00:00 on 2023-07-27!
Thought:{
    "action": "Final Answer",
    "action_input": "Your tour has been scheduled for 11 AM on 2023-07-27. We look forward to showing you around!"
}

> Finished chain.


'Your tour has been scheduled for 11 AM on 2023-07-27. We look forward to showing you around!'

### Test Conversation 3

Asking about an unavailable appointment date.

In [16]:
run_agent("You do have any available appointments on 7/29")



> Entering new AgentExecutor chain...
{
    "action": "appointment_availability",
    "action_input": "7/29"
}
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought:{
    "action": "appointment_availability",
    "action_input": "2022-07-29"
}
Observation: There are no available times on the requested date.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize, but there are no available appointments on July 29th. Is there another date that works for you?"
}

> Finished chain.


'I apologize, but there are no available appointments on July 29th. Is there another date that works for you?'

### Test Conversation 4

Asking about an available appointment time.

In [17]:
run_agent("Can I book a tour for 7/29 at 1pm?")



> Entering new AgentExecutor chain...
{
    "action": "appointment_scheduler",
    "action_input": "2022-07-29 13:00:00"
}
Observation: Appointment could not be booked because appointment day is unavailable.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize, but we don't have any available appointments on July 29th at 1 PM. Is there another date or time that works for you?"
}

> Finished chain.


"I apologize, but we don't have any available appointments on July 29th at 1 PM. Is there another date or time that works for you?"

### Test Conversation 5

Asking about an available appointment time in relative terms.

In [18]:
run_agent("Can I book a tour for this Thursday at 1pm?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Wednesday 2023-07-26 11:23:23
Thought:{
    "action": "appointment_scheduler",
    "action_input": "2023-07-27 13:00:00"
}
Observation: Appointment could not be booked because appointment time is unavailable. Alternative appointment times on that date are 09:00:00, 10:00:00, 11:00:00.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize, but the appointment time you requested on Thursday at 1 PM is unavailable. However, I have alternative appointment times available on that date at 9 AM, 10 AM, and 11 AM. Please let me know if any of these times work for you."
}

> Finished chain.


'I apologize, but the appointment time you requested on Thursday at 1 PM is unavailable. However, I have alternative appointment times available on that date at 9 AM, 10 AM, and 11 AM. Please let me know if any of these times work for you.'

### Test Conversation 6

Asking about an unavailable appointment time.

In [19]:
run_agent("Can I book a tour for 7/30 at 3?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: {
    "action": "appointment_scheduler",
    "

### Test Conversation 7

Asking about an appointment and an inquiry.

In [20]:
run_agent("Can I book a tour for 7/29 at 1pm? i have a car - what is the parking situation like?")



> Entering new AgentExecutor chain...


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4099 tokens. Please reduce the length of the messages.